# Proyecto 03 – Optimización de Cadena de Suministro con IA

Este notebook combina predicción de demanda y optimización logística usando modelos de machine learning y programación lineal.

## 1. Carga de datos

In [ ]:
import pandas as pd

df = pd.read_csv("dataset_logistica.csv")
df.head()

## 2. Predicción de Demanda

In [ ]:
from sklearn.ensemble import RandomForestRegressor

features = ['price', 'promotion']
df = pd.get_dummies(df, columns=['season'], drop_first=True)
X = df[features + [col for col in df.columns if col.startswith("season_")]]
y = df['demand']

model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X, y)

df['predicted_demand'] = model.predict(X)
df[['demand', 'predicted_demand']].head()

## 3. Optimización de Inventario

In [ ]:
from pulp import *

products = df['product'].unique()
order_vars = LpVariable.dicts("Order", products, lowBound=0, cat='Integer')

model_opt = LpProblem("OptimizarInventario", LpMinimize)
costos = {p: np.random.randint(5, 20) for p in products}

model_opt += lpSum([order_vars[p] * costos[p] for p in products])

for p in products:
    model_opt += order_vars[p] >= int(df[df['product'] == p]['predicted_demand'].mean())

model_opt.solve()

resultados = {p: order_vars[p].varValue for p in products}
resultados

## 4. Conclusiones

Se han utilizado técnicas de ML para predecir la demanda y optimizar las órdenes de productos, minimizando costes y ajustándose a las necesidades reales del negocio.